In [2]:
list = [['a', 'b'], ['c'], ['d', 'e']]
for l in list:
  print(len(l))
  if len(l) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symobol in each chunk\nsymbols in chunk: {l}')

2
1


SystemExit: ERROR, Must be more than 1 symobl in each chunk
symbols in chunk: ['c']

c:\Users\ping\MyDrive\py_files\python\py379\.venv\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import yfinance as yf
import pandas as pd
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

In [2]:
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

In [3]:
def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-222]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)

In [4]:
_l_symbols = ['A', 'SHEL', 'YUM']

_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(_l_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
symbols = list(set(_df.columns.get_level_values(0)))
symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

[*********************100%***********************]  3 of 3 completed


In [5]:
close_vs_Yahoo(_l_symbols, _df_adj)

symbol:     A   Date: 2021-12-23   df_Close:  156.79584     Yahoo_Close:  156.79584   %_dif_Close: 0.00000
symbol:  SHEL   Date: 2021-12-23   df_Close:   42.67276     Yahoo_Close:   42.67276   %_dif_Close: 0.00000
symbol:   YUM   Date: 2021-12-23   df_Close:  133.39622     Yahoo_Close:  133.39622   %_dif_Close: 0.00000
No errors found.  Test_symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [ ]:
for symbol in _l_symbols:

  s = _df_adj.iloc[-222]
  # s = df_adjOHLCV.iloc[-222]  # use to check download after FINISHED running all cells
  
  sDate = s.name.strftime('%Y-%m-%d')
  sClose = s[symbol].Close

  df_sym = yf.Ticker(symbol).history(period='2y')
  # 10:45PM got Adj Close
  yhClose = df_sym.loc[sDate]['Close']

  abs_pct_diff = abs(1 - sClose/yhClose)*100
  print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
  Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
  if abs_pct_diff > .0001:
    msg_err = f'{symbol}  %_dif_Close > .0001'
    # raise SystemExit(msg_err)
    print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
  if symbol == _l_symbols[-1]:
    msg_done = f"No errors found.  Test_symbols' 'Adjusted Close' matched Yahoo symbols' Close "
    print(msg_done)

In [ ]:
# # verify df test_symbols' close against Yahoo
# # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
# test_symbols = ['A', 'SHEL', 'YUM']
# _df_list=[]
# _df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
# _df_list.append(_df)
# _df = pd.concat(_df_list, axis=1)
# # get unique symbols from column name
# _l_symbols = list(set(_df.columns.get_level_values(0)))
# _l_symbols.sort()

# _df_adj_list=[]
# # adjust OHLC using 'Adj Close'
# for symbol in _l_symbols:
#   _df1 = auto_adjust(_df[symbol])
#   _df_adj_list.append(_df1)

# _df_adj = pd.concat(_df_adj_list, axis=1)
# _cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# # create multilevel column names
# _col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
# _df_adj.columns = _col_names

# for symbol in _l_symbols:

#   s = _df_adj.iloc[-222]
#   # s = df_adjOHLCV.iloc[-222]  # use to check download after FINISHED running all cells
  
#   sDate = s.name.strftime('%Y-%m-%d')
#   sClose = s[symbol].Close

#   df_sym = yf.Ticker(symbol).history(period='2y')
#   # 10:45PM got Adj Close
#   yhClose = df_sym.loc[sDate]['Close']

#   abs_pct_diff = abs(1 - sClose/yhClose)*100
#   print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
#   Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
#   if abs_pct_diff > .0001:
#     msg_err = f'{symbol}  %_dif_Close > .0001'
#     # raise SystemExit(msg_err)
#     print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
#   if symbol == test_symbols[-1]:
#     msg_done = f"No errors found.  Test_symbols' 'Adjusted Close' matched Yahoo symbols' Close "
#     print(msg_done)